In [ ]:
import moteus

# Differential motor test control

This notebook provides a simple UI to control a 2-motor differential drive (left and right) using sliders:

- Rotation: controls differential steering (turning). Negative = left turn, positive = right turn.
- Position: base forward position applied to both motors (e.g., wheel angle or position).

Usage:
1. Configure `LEFT_ID` and `RIGHT_ID` in the code cell below to match your motor IDs.
2. Start with `dry_run = True` (default) so the notebook only prints computed targets.
3. Verify targets while moving the sliders.
4. When ready, set `dry_run = False`, enable `auto_send`, and use `enable_motors` to send commands to the motors.

Safety:
- Keep `max_pos` conservative until you confirm physical system behaviour.
- Always have an emergency stop or power cutoff available when testing real motors.

Note: The moteus-specific send code is wrapped in a try/except and will fall back to printing if `moteus` is not available or initialization fails. Edit the send wrapper as required for your hardware interface.

In [ ]:
# Helper functions and moteus wrapper
import threading
import time
from dataclasses import dataclass

# Motor IDs (edit to match your hardware)
LEFT_ID = 1
RIGHT_ID = 2

# Safety limits
max_pos = 1.0  # maximum absolute position to send
max_rate = 5.0  # maximum rate

# Default behavior: dry_run prints instead of sending
dry_run = True

# Try to import moteus and create a controller wrapper
_moteus_ctrl = None
try:
    import moteus
    _moteus_adapter = moteus.Controller()
    _moteus_ctrl = _moteus_adapter
except Exception as e:
    _moteus_ctrl = None
    print(f"moteus import or init failed: {e} -- operating in dry-run mode")


def clamp(v, lo, hi):
    return max(lo, min(hi, v))


@dataclass
class DiffTargets:
    left: float
    right: float


def compute_diff_targets(position: float, rotation: float, turn_scale: float = 0.5) -> DiffTargets:
    """Map a base position and rotation to left/right positions.

    position: forward base value applied to both motors
    rotation: -1..1 turning command where sign controls turning direction
    turn_scale: how much rotation affects relative wheel difference
    """
    # simple linear differential: left = position - rotation*turn_scale
    left = position - rotation * turn_scale
    right = position + rotation * turn_scale

    # clamp to safe limits
    left = clamp(left, -max_pos, max_pos)
    right = clamp(right, -max_pos, max_pos)
    return DiffTargets(left, right)


def send_positions(left_pos: float, right_pos: float, left_id=LEFT_ID, right_id=RIGHT_ID):
    """Send position targets to the motors. Uses moteus when available, otherwise prints."""
    left_pos = clamp(left_pos, -max_pos, max_pos)
    right_pos = clamp(right_pos, -max_pos, max_pos)

    if dry_run or _moteus_ctrl is None:
        print(f"DRY RUN -> Left ID {left_id}: {left_pos:.4f}, Right ID {right_id}: {right_pos:.4f}")
        return

    try:
        # Create a batch of position commands
        left_cmd = moteus.Target(position=left_pos)
        right_cmd = moteus.Target(position=right_pos)
        # Send both commands in one packet if the adapter supports it
        # For moteus, send_commands accepts a list of (id, command)
        _moteus_ctrl.set_positions([(left_id, left_cmd), (right_id, right_cmd)])
    except Exception as e:
        print(f"Failed to send moteus commands: {e}")


# Background sender thread management
_sender_thread = None
_sender_stop = threading.Event()


def start_sender(get_state_fn, interval=0.05, auto_send=False):
    """Start a background thread that polls get_state_fn -> (position, rotation) and sends positions when auto_send True."""
    global _sender_thread, _sender_stop
    if _sender_thread is not None and _sender_thread.is_alive():
        return _sender_thread

    _sender_stop.clear()

    def _run():
        while not _sender_stop.is_set():
            pos, rot, auto = get_state_fn()
            targets = compute_diff_targets(pos, rot)
            if auto:
                send_positions(targets.left, targets.right)
            else:
                # print computed targets once every 0.2s to avoid flooding
                print(f"Computed -> left: {targets.left:.4f}, right: {targets.right:.4f}")
                time.sleep(0.2)
            time.sleep(interval)

    t = threading.Thread(target=_run, daemon=True)
    _sender_thread = t
    t.start()
    return t


def stop_sender():
    global _sender_stop, _sender_thread
    _sender_stop.set()
    if _sender_thread is not None:
        _sender_thread.join(timeout=1.0)
    _sender_thread = None

print("Helper functions loaded. Configure LEFT_ID/RIGHT_ID and set dry_run=False to enable live sending.")

In [ ]:
# UI: sliders and controls
import ipywidgets as widgets
from IPython.display import display, clear_output

# State variables
_state = {"position": 0.0, "rotation": 0.0, "auto_send": False, "enable_motors": False}

position_slider = widgets.FloatSlider(value=0.0, min=-max_pos, max=max_pos, step=0.01, description='Position')
rotation_slider = widgets.FloatSlider(value=0.0, min=-1.0, max=1.0, step=0.01, description='Rotation')
auto_toggle = widgets.ToggleButton(value=False, description='Auto send')
enable_toggle = widgets.Checkbox(value=False, description='Enable motors (live)')
left_id_box = widgets.BoundedIntText(value=LEFT_ID, min=0, max=255, description='Left ID')
right_id_box = widgets.BoundedIntText(value=RIGHT_ID, min=0, max=255, description='Right ID')
dry_run_toggle = widgets.Checkbox(value=True, description='Dry run (no send)')
start_button = widgets.Button(description='Start background')
stop_button = widgets.Button(description='Stop background')

out = widgets.Output()


def _get_state():
    # return position, rotation, and auto flag
    _state['position'] = float(position_slider.value)
    _state['rotation'] = float(rotation_slider.value)
    _state['auto_send'] = bool(auto_toggle.value)
    _state['enable_motors'] = bool(enable_toggle.value)
    # propagate IDs and dry_run
    globals()['LEFT_ID'] = int(left_id_box.value)
    globals()['RIGHT_ID'] = int(right_id_box.value)
    globals()['dry_run'] = bool(dry_run_toggle.value)
    return _state['position'], _state['rotation'], _state['auto_send']


def _update_display(change=None):
    pos, rot, _ = _get_state()
    targets = compute_diff_targets(pos, rot)
    with out:
        clear_output(wait=True)
        print(f"Position: {pos:.3f}, Rotation: {rot:.3f}")
        print(f"Targets -> left: {targets.left:.4f}, right: {targets.right:.4f}")


position_slider.observe(_update_display, names='value')
rotation_slider.observe(_update_display, names='value')


def _start_clicked(b):
    start_sender(_get_state, interval=0.05, auto_send=auto_toggle.value)


def _stop_clicked(b):
    stop_sender()


start_button.on_click(_start_clicked)
stop_button.on_click(_stop_clicked)

# Layout
controls = widgets.VBox([
    widgets.HBox([position_slider, rotation_slider]),
    widgets.HBox([auto_toggle, enable_toggle, dry_run_toggle]),
    widgets.HBox([left_id_box, right_id_box]),
    widgets.HBox([start_button, stop_button])
])

_display = widgets.VBox([controls, out])

display(_display)
_update_display()  # initial show

print('UI ready. Use Start background to begin updating targets. Set Dry run off and Enable motors to attempt real sends.')

In [ ]:
# Quick demo: compute a few sample targets
examples = [(-0.5, -0.5), (0.0, 0.0), (0.5, 0.5), (0.0, 1.0)]
print('Sample (rotation, position) -> (left, right)')
for rot, pos in examples:
    t = compute_diff_targets(pos, rot)
    print(f"rot={rot}, pos={pos} -> left={t.left:.4f}, right={t.right:.4f}")

print('\nWhen ready:\n - Clear Dry run checkbox to allow real sends (ensure motors and IDs are correct).\n - Click Start background to begin sender thread.\n - Toggle Auto send to have continuous sends from the background thread.')